In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import zipfile
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", 'r') as zipp:
    zipp.extractall("/kaggle/files/")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", 'r') as zipp:
    zipp.extractall("/kaggle/files/")
    
os.listdir("/kaggle/files/")

In [ ]:
train_img = os.listdir("/kaggle/files/train/")
test_img = os.listdir("/kaggle/files/test1/")

#print(train_image[0].split("."))
files = []
classes = []
for file in train_img:
    files.append(file)
    if file.startswith("dog"):
        classes.append("1")
    else:
        classes.append("0")
print(classes[:10])

d_train = pd.DataFrame({
    "file":train_img,
    "class":classes
})
d_train = d_train.reset_index(drop=True)
print(d_train)

In [ ]:
#print(test_image)

In [ ]:
x_train, x_val = train_test_split(d_train, test_size=0.33, random_state = 42)
x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
#print(x_val)
#print(x_val.iloc[0,0])
x_valImg = plt.imread("/kaggle/files/train/"+x_val.iloc[0,0])
#print(x_valImg[0])
plt.imshow(x_valImg, cmap=plt.cm.binary)
plt.show()
print(x_val['class'])

In [ ]:
x_trainImg = plt.imread("/kaggle/files/train/"+x_train.iloc[0,0])
print(x_trainImg[0])
plt.imshow(x_trainImg, cmap = plt.cm.binary)
plt.show()
print(x_train.shape)
print(x_train["class"])

In [ ]:
train_datagenerator=ImageDataGenerator(
            horizontal_flip=True,
            rescale=1./255
)

valid_datagenerator=ImageDataGenerator(
            rescale=1./255
)

train_generator = train_datagenerator.flow_from_dataframe(
            x_train,
            "/kaggle/files/train/",
            x_col="file",
            y_col="class",
            target_size=(224,224),
            class_mode="binary"

)

valid_generator=valid_datagenerator.flow_from_dataframe(
            x_val,
            "/kaggle/files/train/",
            x_col="file",
            y_col="class",
            target_size=(224,224),
            class_mode="binary"
)
print(type(valid_generator))

In [ ]:
image, labels = next(train_generator)
print(image.shape)
print(labels.shape)

In [ ]:
def plotImages(image):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for ax, img in zip(axes, image):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

plotImages(image)
print(labels[:6])

In [ ]:
img = plt.imread("/kaggle/files/train/dog.12493.jpg")
print(img.shape)

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), padding='same',input_shape=(224,224,3), activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
hist = model.fit_generator(generator=train_generator, validation_data=valid_generator, epochs=5, steps_per_epoch=50)

In [ ]:
hist.history

In [ ]:
x_test_col = []
for file in test_img:
    x_test_col.append(file)

x_test = pd.DataFrame({'filename': x_test_col})
test_generator = valid_datagenerator.flow_from_dataframe(x_test,
                                             directory = "/kaggle/files/test1",
                                             x_col = "filename",
                                             y_col = None,
                                             class_mode = None,
                                             target_size = (224,224))

In [ ]:
predictions=model.predict(test_generator)
predictions.size

In [ ]:
print(type(predictions))
print(len(predictions))
i = random.randint(0,len(predictions))
print(predictions[i])
if predictions[i] > 0.5:
    print("Dog")
else:
    print("Cat")

image = plt.imread("/kaggle/files/test1/"+test_img[i])

plt.imshow(image, cmap=plt.cm.binary)
plt.show()

In [ ]:
pred = []
for i in range(len(predictions)):
    if predictions[i] > 0.5:
        pred.append(1)
    else:
        pred.append(0)


In [ ]:
output = pd.DataFrame({"Id": np.arange(1,12501), "label": pred})
output.to_csv("/kaggle/working/submission.csv", index = False)

print(output)